In [2]:

import pandas as pd
import matplotlib.pyplot as plt
import sqlite3 as lite
from sqlite3 import Error
from pathlib import Path
from datetime import date
import numpy as np
# import seaborn as sns
import matplotlib.ticker as tick
import requests
import difflib as diff
import re
import csv
import ast
import math
import os
import random
from codetf.code_utility.apex.apex_code_utility import ApexCodeUtility

apex_code_utility = ApexCodeUtility()
# from pandarallel import pandarallel

/hpcfs/users/a1232991/local/virtualenvs/llm/lib/python3.9/site-packages/codetf/tree-sitter-prebuilts/Linux


In [57]:
language = "Python"
# language = "TypeScript"

In [58]:
def create_connection(db_file):
    """
    create a connection to sqlite3 database
    """
    conn = None
    try:
        conn = lite.connect(db_file, timeout=10)  # connection via sqlite3
    except Error as e:
        print(e)
    return conn

In [59]:
# ['repo_url', 'hash', 'committer', 'committer_date', 'filename',
    #    'programming_language', 'signature', 'code', 'label', 'before_change',
    #    'added_only']
short_name = {"JavaScript": "JS", "Python": "PY", "TypeScript": "TS", "CPP": "CPP", "C": "C", "Java": "Java", "C#": "C#"}
sname = short_name[language]

df_codes = pd.read_parquet(f'/hpcfs/users/a1232991/Data/CVESingle/CVE_{sname}/{sname.lower()}_data.parquet', engine='pyarrow')
df_codes = df_codes[df_codes['before_change']=='True']
df_codes = df_codes[["repo_url", "hash", "code", "label"]]
df_codes.columns = ['repo_url', 'hash', 'code_before', 'target']

In [60]:
df_codes.head()

,repo_url,hash,code_before,target
0,https://github.com/latchset/kdcproxy.git,f274aa6787cb8b3ec1cc12c440a56665b7231882,"def __call__(self, env, start_response):\n...",1
1,https://github.com/latchset/kdcproxy.git,f274aa6787cb8b3ec1cc12c440a56665b7231882,"def __filter_addr(self, addr):\n if...",0
2,https://github.com/latchset/kdcproxy.git,f274aa6787cb8b3ec1cc12c440a56665b7231882,"def sock_type(self, sock):\n try:\n...",0
3,https://github.com/latchset/kdcproxy.git,f274aa6787cb8b3ec1cc12c440a56665b7231882,def __init__(self):\n self.__resolv...,0
4,https://github.com/latchset/kdcproxy.git,f274aa6787cb8b3ec1cc12c440a56665b7231882,"def __str__(self):\n return ""%d %s""...",0


## Remove duplicate codes

In [61]:
print("Before drop duplicates: ", df_codes.shape)

Before drop duplicates:  (15217, 4)


In [62]:
# Sort by 'label' in descending order
df_codes = df_codes.sort_values(by='target', ascending=False)

df_codes.drop_duplicates(subset=['code_before'], keep='first', inplace=True)

In [63]:
print("After drop duplicates: ", df_codes.shape)

After drop duplicates:  (13419, 4)


## Remove comments in the code

In [64]:
df_codes['code_before'] = df_codes['code_before'].apply(lambda x: apex_code_utility.remove_comments(x))

In [3]:
code = 'def iter_fields(node):\
    """"""\
    Yield a tuple of ``(fieldname, value)`` for each field in ``node._fields``\
    that is present on *node*.\
    """"""\
    for field in node._fields:\
        try:\
            yield field, getattr(node, field)\
        except AttributeError:\
            pass'

after_code = apex_code_utility.remove_comments(code)

In [4]:
print("After remove comments: ", after_code)

After remove comments:  def iter_fields(node):    """"""    Yield a tuple of ``(fieldname, value)`` for each field in ``node._fields``    that is present on *node*.    """"""    for field in node._fields:        try:            yield field, getattr(node, field)        except AttributeError:            pass


In [65]:
df_codes.target.value_counts()

target
0    12442
1      977
Name: count, dtype: int64

## Format the code

In [66]:
# Not in use
import re

def code_format(row):
    name = f"{random.getrandbits(128)}.py"
    x = x.strip()
    with open(name, 'w') as f:
        f.write(x)
    
    # execute black through command line
    os.system(f"black {name}")

    # read back the formatted code
    with open(name, 'r') as f:
        x = f.read()
    os.remove(name)
    return x

def replace_whitespace(text):
    text = text.strip()
    return re.sub(r'\s+', ' ', text)

In [67]:
# if language == 'Python':
#     # pandarallel.initialize(progress_bar=True)
#     # df_codes['code_before'] = df_codes['code_before'].apply(code_format)
#     df_codes['code_before'] = df_codes['code_before'].apply(replace_whitespace)
df_codes['code_before'] = df_codes['code_before'].apply(lambda x: x.strip())

## Show the tokenized statistics

In [68]:
# Check the length of the code after tokenization
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
token_lengths = df_codes['code_before'].apply(lambda x: len(tokenizer.tokenize(x)))
token_lengths.describe()

Token indices sequence length is longer than the specified maximum sequence length for this model (2408 > 512). Running this sequence through the model will result in indexing errors


count    13419.000000
mean       326.884045
std        634.483865
min          9.000000
25%         64.000000
50%        147.000000
75%        349.000000
max      18221.000000
Name: code_before, dtype: float64

In [69]:
df_codes.iloc[0]['code_before']

'def __call__(self, env, start_response):\n        try:\n            # Validate the method\n            method = env["REQUEST_METHOD"].upper()\n            if method != "POST":\n                raise HTTPException(405, "Method not allowed (%s)." % method)\n\n            # Parse the request\n            try:\n                length = int(env["CONTENT_LENGTH"])\n            except AttributeError:\n                length = -1\n            try:\n                pr = codec.decode(env["wsgi.input"].read(length))\n            except codec.ParsingError as e:\n                raise HTTPException(400, e.message)\n\n            # Find the remote proxy\n            servers = self.__resolver.lookup(\n                pr.realm,\n                kpasswd=isinstance(pr, codec.KPASSWDProxyRequest)\n            )\n            if not servers:\n                raise HTTPException(503, "Can\'t find remote (%s)." % pr)\n\n            # Contact the remote server\n            reply = None\n            wsocks = 

In [70]:
df_codes.iloc[0]['repo_url'] + '/commit/'+df_codes.iloc[0]['hash']

'https://github.com/latchset/kdcproxy.git/commit/f274aa6787cb8b3ec1cc12c440a56665b7231882'

In [71]:
df_codes['commit_url'] = df_codes.apply(lambda x: x['repo_url'] + '/commit/'+x['hash'], axis=1)

In [72]:
counts = df_codes.target.value_counts()

In [73]:
ratio = counts['0']/counts['1']

In [74]:
ratio = int(ratio)

In [75]:
df_codes.isna().sum().sum()

0

In [76]:
# convert label to int
df_codes['target'] = df_codes['target'].astype(int)

In [77]:
if 'flaw_lines' in df_codes.columns:
    df_flaw_index = df_codes.apply(
        lambda row: ",".join([str(int(i) - int(row['start_line'])) for i in row['flaw_lines'].split(",")])
                    if row['target'] == 1 else -1,
        axis=1
    )

    # Update the flaw_index column in the DataFrame
    df_codes['flaw_index'] = df_flaw_index
else:
    df_codes['flaw_index'] = -1
    df_codes['flaw_lines'] = -1

In [78]:
from sklearn.model_selection import train_test_split

os.makedirs("data", exist_ok=True)
df_python_align = df_codes[['code_before', 'target', 'flaw_lines', 'flaw_index', 'commit_url']]
df_python_align.columns = ['processed_func', 'target', 'flaw_line', 'flaw_line_index', 'commit_url']

# Split the DataFrame into train and remaining
train_df, remaining_df = train_test_split(df_python_align, test_size=0.2, random_state=42)

# Split the remaining DataFrame into validation and test
val_df, test_df = train_test_split(remaining_df, test_size=0.5, random_state=42)

# Print the shapes of the resulting DataFrames
print("Train set shape:", train_df.shape)
print("Validation set shape:", val_df.shape)
print("Test set shape:", test_df.shape)

train_df.to_csv(f"/hpcfs/users/a1232991/Data/CVESingle/CVE{language}_train.csv", index=None)
val_df.to_csv(f"/hpcfs/users/a1232991/Data/CVESingle/CVE{language}_val.csv", index=None)
test_df.to_csv(f"/hpcfs/users/a1232991/Data/CVESingle/CVE{language}_test.csv", index=None)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Train set shape: (10735, 5)
Validation set shape: (1342, 5)
Test set shape: (1342, 5)
